# 2024 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [99]:
import nltk
import pandas as pd
import numpy as np
import json
import ijson


In [100]:
# 3min to process
# 自己定义地址
pure_evidence_data = pd.read_json("C:/Users/Administrator/Desktop/UNI/Master/NLP/Proj/evidence.json", lines=True)
    

In [101]:
pure_dev_data_set = pd.read_json("data/dev-claims.json")
pure_train_data_set = pd.read_json("data/train-claims.json")
pure_test_data_set = pd.read_json("data/test-claims-unlabelled.json")
pure_dev_baseline_date_set = pd.read_json("data/dev-claims-baseline.json")

In [102]:
def structurelise_data_set(data_set):
    structured_data_set = []
    for claim_id, claim_info in data_set.items():
        claim_text = claim_info['claim_text']
        claim_label = ''
        if 'claim_label' in claim_info:
            claim_label = claim_info['claim_label']
        evidence_ids = []
        if 'evidences' in claim_info:
            evidence_ids = claim_info['evidences']
        evidence_texts = []
        for evidence_id in evidence_ids:
            evidence_texts.append(pure_evidence_data[evidence_id][0])
        structured_data_set.append({
            'claim_id' : claim_id,
            'claim_text': claim_text,
            'evidence_texts': evidence_texts,
            'label': claim_label
        })
    return structured_data_set

structured_dev_data_set = structurelise_data_set(pure_dev_data_set)
structured_train_data_set = structurelise_data_set(pure_train_data_set)
structured_test_data_set = structurelise_data_set(pure_test_data_set)
structured_dev_baseline_date_set = structurelise_data_set(pure_dev_baseline_date_set)
print(structured_train_data_set[0:5])
    

[{'claim_id': 'claim-1937', 'claim_text': 'Not only is there no scientific evidence that CO2 is a pollutant, higher CO2 concentrations actually help ecosystems support more plant and animal life.', 'evidence_texts': ['At very high concentrations (100 times atmospheric concentration, or greater), carbon dioxide can be toxic to animal life, so raising the concentration to 10,000 ppm (1%) or higher for several hours will eliminate pests such as whiteflies and spider mites in a greenhouse.', 'Plants can grow as much as 50 percent faster in concentrations of 1,000 ppm CO 2 when compared with ambient conditions, though this assumes no change in climate and no limitation on other nutrients.', 'Higher carbon dioxide concentrations will favourably affect plant growth and demand for water.'], 'label': 'DISPUTED'}, {'claim_id': 'claim-126', 'claim_text': 'El Niño drove record highs in global temperatures suggesting rise may not be down to man-made emissions.', 'evidence_texts': ['While ‘climate c

In [103]:
import re
import nltk
from nltk.corpus import stopwords

nltk.download('wordnet')
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

def text_preprocessing(text):
    words = text.lower().split()  
    
    new_words = []
    for w in words:
        w = lemmatize(w)
        if w not in stopwords:
            new_words.append(w)

    processed_text = " ".join(new_words)
    return processed_text    

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    words = text.split()
    new_words = []
    for word in words:
        word = lemmatize(word)
        if word not in stopwords:
            new_words.append(word)
    processed_text = " ".join(new_words)
    return processed_text

def preprocess_data_set(data_set):
    for item in data_set:
        item['claim_text'] = preprocess_text(item['claim_text'])
        item['evidence_texts'] = [preprocess_text(evidence) for evidence in item['evidence_texts']]
    return data_set

processed_dev_data_set = preprocess_data_set(structured_dev_data_set)
processed_train_data_set = preprocess_data_set(structured_train_data_set)
processed_test_data_set = preprocess_data_set(structured_test_data_set)
processed_dev_baseline_date_set = preprocess_data_set(structured_dev_baseline_date_set)
print(processed_train_data_set[0:5])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[{'claim_id': 'claim-1937', 'claim_text': 'scientific evidence co2 pollutant higher co2 concentration actually help ecosystem support plant animal life', 'evidence_texts': ['high concentration 100 time atmospheric concentration greater carbon dioxide toxic animal life raise concentration 10000 ppm 1 higher several hour eliminate pest whitefly spider mite greenhouse', 'plant grow much 50 percent faster concentration 1000 ppm co 2 compare ambient condition though assume change climate limitation nutrient', 'higher carbon dioxide concentration favourably affect plant growth demand water'], 'label': 'DISPUTED'}, {'claim_id': 'claim-126', 'claim_text': 'el nio drive record high global temperature suggest rise may manmade emission', 'evidence_texts': ['climate change due natural force human activity substantial evidence indicate human activity specifically increase greenhouse gas ghgs emission key factor pace extent global temperature increase', 'acceleration due mostly humancaused global wa

In [104]:
structured_evidence_data = []
for evidence_id, evidence_text  in pure_evidence_data.items():
    structured_evidence_data.append({
        'evidence_id' : evidence_id,
        'evidence_text': evidence_text[0],
    })
for item in structured_evidence_data:
    item['evidence_text'] = preprocess_text(str(item['evidence_text']))

In [105]:
print(len(structured_evidence_data))
print(structured_evidence_data[1208826])
print(len(structured_train_data_set))
print(len(structured_dev_data_set))

1208827
{'evidence_id': 'evidence-1208826', 'evidence_text': 'storyline revolve around giant plesiosaur akin loch ness monster appear crater lake northern california near susanville confuse much famous crater lake oregon'}
1228
154


In [106]:
longest_sentence = 0
longest_sentence_text = ""
for item in structured_evidence_data:
    if len(item['evidence_text'].split())> longest_sentence:
        longest_sentence = len(item['evidence_text'].split())
        longest_sentence_text = item['evidence_text']
print(longest_sentence)
print(longest_sentence_text)

355
58 year political life arrest 17 time participate gandhiji satyagraha kisan mazdoor harijan student movement actually spend total 13 year various jail different occasion sentence 10 year rigorous imprisonment gorakhpur conspiracy case august 1942 movement keep condemn cell 26 month 1942 44 shoot august rebellion august 6 1942 receive bullet injury nepal freedom struggle december 21 1950 president samajwadi congress party uttar pradesh ii tax satyagraha compaign committee iii kanpur district congress committee 1930 33 iv gorakhpur district congress committee 1937 52 v kisan mazdoor praja party 1952 60 member uttar pradesh congress committee india congress committee 1928 51 ii executive uttar pradesh pradesh congress committee 1937 52 iii uttar pradesh bihar sugar control board many year since 1937 iv sugar labour inquiry committee khaitan committee 1938 40 v executive council sanskrit university varanasi president allindia sugarcane grower association ii unite chini mill mazdoor fed

Further data process

In [107]:
from collections import Counter

def tokenize(text):
    return text.split()

# global word count
word_count = Counter()
for claim in processed_train_data_set:
    word_count.update(tokenize(claim['claim_text']))
for evidence in structured_evidence_data:
    word_count.update(tokenize(evidence['evidence_text']))

sorted_word_count = {word: count for word, count in sorted(word_count.items(), key=lambda x : x[1], reverse=True)}

# create vocab with index
vocabulary = {'<PAD>' : 0, '<CLS>': 1, '<UNK>' : 2, '<SEP>': 3}
vocabulary.update({word: idx + 4 for idx, word in enumerate(sorted_word_count.keys())})


In [108]:
print(list(sorted_word_count.items())[0:10])
print(list(vocabulary.items())[:10])

[('also', 66990), ('state', 57325), ('bear', 54596), ('first', 50702), ('one', 47006), ('new', 43508), ('include', 39604), ('play', 38769), ('use', 38202), ('year', 38197)]
[('<PAD>', 0), ('<CLS>', 1), ('<UNK>', 2), ('<SEP>', 3), ('also', 4), ('state', 5), ('bear', 6), ('first', 7), ('one', 8), ('new', 9)]


In [109]:
print(len(vocabulary))

644056


In [110]:
label2index = {'SUPPORTS':0, 'NOT_ENOUGH_INFO':1, 'REFUTES':2, 'DISPUTED':3}
index2label = {0:'SUPPORTS', 1:'NOT_ENOUGH_INFO', 2:'REFUTES', 3:'DISPUTED'}

In [111]:
def text_to_sequence(text, vocab, max_length):
    tokens = tokenize(text)
    token_ids = [vocab.get(token, vocab['<UNK>']) for token in tokens if token in vocab]

    if (len(token_ids) > max_length):
        token_ids = token_ids[:max_length]
    else :
        token_ids += [vocabulary['<PAD>']] * (max_length - len(token_ids))
    return token_ids

In [112]:
import torch
from torch.utils.data import Dataset, DataLoader

class ClaimCheckDataset(Dataset):
    def __init__(self, data, vocabulary, max_length):
        self.data = data
        self.vocabulary = vocabulary
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data[index]
        claim_text = text_to_sequence(item["claim_text"], self.vocabulary, self.max_length)
        evidence_text = ' <SEP> '.join(item['evidence_texts'])
        evidence_text = text_to_sequence(evidence_text, self.vocabulary, self.max_length)
        label = label2index[item['label']]
        return {
            'claim_text': torch.tensor(claim_text, dtype=torch.long),
            'evidence_text': torch.tensor(evidence_text, dtype=torch.long),
            'label': torch.tensor(label, dtype=torch.long)
        }
train_dataset = ClaimCheckDataset(processed_train_data_set, vocabulary, max_length=512)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [113]:
validate_dataset = ClaimCheckDataset(processed_dev_data_set, vocabulary, max_length=512)
validate_loader = DataLoader(validate_dataset, batch_size=32, shuffle=True)


# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [114]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class TransformerModel(nn.Module):
    def __init__(self, num_vocab, num_dimension, num_head, num_hidden, num_layer, num_category, max_length):
        super(TransformerModel, self).__init__()
        
        self.num_hidden = num_hidden
        # transfrom word into vector through embed
        self.embed = nn.Embedding(num_vocab, num_dimension)
        # transfrom sequence into vector
        self.pos_embed = nn.Embedding(max_length, num_dimension)
        encoder_layer = nn.TransformerEncoderLayer(d_model=num_dimension, nhead=num_head, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer=encoder_layer, num_layers=num_layer, norm=nn.LayerNorm(num_hidden))
        self.hidden_layer = nn.Linear(num_hidden, num_hidden // 2)
        self.classifier = nn.Linear(num_hidden // 2, num_category)


    def forward(self, text):
        mask = text == 0
        batch_size, seq_length = text.size()
        position = torch.arange(seq_length, device=text.device).unsqueeze(0).expand(batch_size, -1)
        text_x = self.embed(text) + self.pos_embed(position)
        x_encoded = self.transformer_encoder(text_x, src_key_padding_mask = mask)
        x_pooled = x_encoded.mean(dim=1)
        layer_x = torch.tanh(self.hidden_layer(x_pooled))
        label_x = self.classifier(layer_x)
        return label_x



In [115]:
transformer_model = TransformerModel(num_vocab=len(vocabulary), num_dimension=124, num_head=4, num_hidden=124, num_layer=3, num_category = 4, max_length=512)


In [116]:
inputs = torch.tensor([1088, 28287, 2377, 2004, 1748]).unsqueeze(0)
output = transformer_model(inputs)
print(output)

tensor([[ 0.3518,  0.1872, -0.2252, -0.0151]], grad_fn=<AddmmBackward0>)


In [117]:
probabilities = torch.softmax(output, dim=1)
print(probabilities)

tensor([[0.3223, 0.2734, 0.1810, 0.2233]], grad_fn=<SoftmaxBackward0>)


In [118]:
predicted_class = torch.argmax(probabilities, dim=1)
print(predicted_class)

tensor([0])


In [119]:
#Train

# 定义损失函数
criterion = nn.CrossEntropyLoss()

# 定义优化器
optimizer = optim.Adam(transformer_model.parameters(), lr=0.001)

def train_model(model, train_loader, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        model.train()  # 设置模型为训练模式
        total_loss = 0
        for batch in train_loader:
            claims, evidences, labels = batch['claim_text'], batch['evidence_text'], batch['label']
            # 清空优化器
            optimizer.zero_grad()
            
            # 前向传播
            outputs = model(claims)  # 确保你的模型和数据加载方式是匹配的
            loss = criterion(outputs, labels)
            
            # 反向传播
            loss.backward()
            
            # 参数更新
            optimizer.step()
            
            total_loss += loss.item()

            # 前向传播
            outputs = model(evidences)  # 确保你的模型和数据加载方式是匹配的
            loss = criterion(outputs, labels)
            
            # 反向传播
            loss.backward()
            
            # 参数更新
            optimizer.step()
            
            total_loss += loss.item()
        
        # 打印每个epoch的损失
        print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}')

train_model(transformer_model, train_loader, criterion, optimizer)

Epoch 1, Loss: 2.54614268663602
Epoch 2, Loss: 2.541097864126548
Epoch 3, Loss: 2.533978880980076
Epoch 4, Loss: 2.533428213535211
Epoch 5, Loss: 2.533907801677019


In [120]:
#Eval
from sklearn.metrics import f1_score
def calculate_accuracy(y_pred, y_true):
    prediction = torch.argmax(y_pred, dim=1)
    correct = (prediction == y_true).float()
    accuracy = correct.sum() / len(correct)
    return accuracy
def calculate_f1(y_pred, y_true, average = "macro"):
    # 将预测结果转换为类别索引
    y_pred = torch.argmax(y_pred, dim=1).cpu().numpy()
    y_true = y_true.cpu().numpy()
    # 使用 sklearn 的 f1_score 函数计算 F1 分数
    return f1_score(y_true, y_pred, average=average)




def evaluate_model(model, valid_loader, criterion):
    model.eval()  # 设置模型为评估模式
    total_loss = 0
    total_accuracy = 0
    total_f1 = 0
    num_batch = 0
    with torch.no_grad():
        for batch in valid_loader:
            claims, evidences, labels = batch['claim_text'], batch['evidence_text'], batch['label']
            outputs = model(claims)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total_accuracy += calculate_accuracy(output, labels)
            total_f1 += calculate_f1(outputs, labels)
            num_batch += 1
    
    print(f'Validation Loss: {total_loss/len(valid_loader)}')
    print(f'Validation F1 Score: {total_f1 / num_batch}')
    print(f'Validation Accuracy: {total_accuracy / len(valid_loader)}')
evaluate_model(transformer_model, validate_loader, criterion)


Validation Loss: 1.365264630317688
Validation F1 Score: 0.10228011329755515
Validation Accuracy: 0.4451923370361328


# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*